In [2]:
import pandas as pd
import requests
from datetime import datetime
import calendar
import matplotlib.pyplot as plt
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
import math
import time

from utils.utils import setLinkEod, setLinkIntd, setLinkRatios, setLinkIncomeStatement, getBuyDay, getTradingDay, withinQuarter, usBday

In [33]:
contracts = pd.read_csv("data/portfolioTesting/c23.csv")
contracts['Start Date']= pd.to_datetime(contracts['Start Date'])
contracts['End Date'] = pd.to_datetime(contracts['End Date'])
contracts=contracts.drop_duplicates(subset='Award ID').sort_values(by='Start Date')
contracts['Weight'] = contracts.groupby(['Recipient Name', 'Start Date'])['Recipient Name'].transform('count')
contracts['Annualized Value'] = (contracts['Award Amount'] / (contracts['End Date'] - contracts['Start Date']).dt.days) * 365

In [4]:
from dataclasses import dataclass 
import uuid


# datastructure to track trades made on the same underlying via contractID 
@dataclass
class transaction: 
    contractID: int
    num: int
    pps: int

# globally unique id generator 
class uniqueId:
    def __init__(self):
        self.id = str(uuid.uuid4())

In [5]:
from enum import Enum

# enum to abstract cacheFundementals 
class FundementalIndicator(Enum):
    FINANCIAL_RATIOS = "financial ratios"
    FINANCIAL_STATEMENT = "financial statement"
    INCOME_STATEMENT = "income statement"
    BALANCE_SHEET = "balance sheet"

In [6]:
class BenchmarkPortfolio:
    """ a benchmarked portfolio that mimicks all trades from the strategy using the SP500 
        
        @c: starting capital 
        @w: percentage of capital allocated per trade 

        id: a unique 6 digit number to keep track of the portfolio when multithreading
        buys: a list of trades made on the SP500 
        valuation: tracks the value of the portfolio after each transaction 
        holds: flag to track if any pending assets need to be sold before next purcahse
        bankrupt: well... if the portfolio has sufficient funds to continue
    """

    def __init__(self, c, w):
        self.id = uniqueId().id
        self.capital = c
        self.weight = w
        self.trades = []
        self.valuation = [c]
        self.exceptions = []
        self.fundementals = {} #hashmap <String (tick), HashMap<String (indicator), LinkedList>() (reports); 
        self.holds = False
        self.bankrupt = False
        
    def buy(self, pps, shares, id, buyDate):
        if self.capital >= pps*shares: 
            if self.holds == False:
                self.capital -= pps*shares
                self.trades.append(transaction(id, shares, pps))
                self.valuation.append(self.capital)
                self.holds = True 
                print(f"B{shares}{'SPY'}{pps}D{buyDate}$${self.capital}")

                return True 
            else: 
                print ("Buy transaction for SPY at {shares} per share on {buyDate} cancelled - there are pending market positions to be cleared")
                return False 
        else: 
            print("insufficient capital to complete buy transaction")
            self.bankrupt = True   
            return False 

    def sell(self, pps, id, sellDate): 
        for transaction in (self.trades):
            if transaction.contractID == id: 
                self.capital += transaction.num * pps 
                self.valuation.append(self.capital)
                self.trades.remove(transaction) 
                print(f"S{transaction.num}{'SPY'}{pps}D{sellDate}$--{self.capital}")
                break 
    
    def clearTradeSPY(self, id, date):
        """ clears unsold SPY asset (id) from the book using the intial purchase price 
        """
        for transaction in (self.trades):
            if transaction.contractID == id: 
                self.capital += transaction.num * transaction.pps
                self.valuation.append(self.capital)
                self.trades.remove(transaction)
                print(f"FS{transaction.num}SPY{transaction.pps}D{date}$--{self.capital}")
                break

    def sellSuccessful(self):
        """ toggles the hold flag when a sell has been executed 
        """
        self.holds = False

    def isClear (self):
        """ checks if any trades are pending sales 
        """
        return not self.holds

    def logError (self, e, t, d, id, isSale, func): 
        """ logs exceptions in the portfolio's exceptions list and prints to CLI

            @e: the exception that was raised
            @t: the ticker of the underlying asset
            @d: the date of the transaction
            @id: the contractID that was being processed
            @isSale: boolean flag to indicate if the error was on a sell order
        """
        if isSale: 
            log = f"Unexpected error: {str(e)} when trying to sell at {func} for {t} on {d} via contractID: {id}"
        else:
            log = f"Unexpected error: {str(e)} on trying to buy at {func} for {t} on {d} via contractID: {id}"
        self.exceptions.append(log)
        print(log)

    
    def cacheIndicator (self, indicator: FundementalIndicator, dataframe, tick):
        """ appends the API-request dataframe of {tick}'s {indicator} to self.fundementals (dict) for o(1) access later
            
            @indicator: the type of technical indicator being logged
            @dataframe: the dataframe containing the indicator data
            @tick: the ticker of the underlying company
        """
        assert len(dataframe)!=0, f"cannot log an empty dataframe for {tick}'s {indicator.value[0]}"

        if tick not in self.fundementals:
            self.fundementals[tick] = {indicator.value, [dataframe]}

        elif indicator not in self.fundementals[tick]:
            self.fundementals[tick][indicator.value] = [dataframe]

        else: 
            self.fundementals[tick][indicator.value].append(dataframe) 
        

    def getIndicatorReport (self, indicator: FundementalIndicator, tick, date):
        """ retrieves the most recent reported quarter of {tick}'s {indicator.value} if it's within range (no newer statements have been released)
            otherwise []
        """
        if tick not in self.fundementals or indicator.value not in self.fundementals[tick]:
            return None
        else: 
            recentCachedReport = self.fundementals[tick][indicator.value][-1]
            reportedDate = recentCachedReport['date']
            if (withinQuarter(reportedDate, date)):
                return recentCachedReport
            else:
                return None

In [7]:
class Portfolio(BenchmarkPortfolio) : 
    """ a simulated portfolio that automatcally purchases and sells 
        assets using the government contracts + MR strategy 

        @z: mean reversion z-score treshold to sell
        @w: percentage of capital allocated to each trade 
        @c: starting capital

        trades: a dictionary to track all trades made on the same underlying
    """
    def __init__(self, z, w, c):
        super().__init__(c, w)
        self.trades = {}
        self.mRThresh = z

    def logTrade(self, ticker, id, shares, pps): 
        if ticker not in self.trades: 
            self.trades[ticker] = [transaction(id, shares, pps)]
        else: 
            self.trades[ticker].append(transaction(id, shares, pps))
                        
    def buy(self, ticker, pps, shares, id, buyDate): 
        if self.capital >= pps*shares: 
            self.capital -= pps*shares
            self.logTrade(ticker, id, shares, pps)
            #self.buys.append([transaction(id, shares), buyDate])
            self.holds = True
            self.valuation.append(self.capital)
            print(f"B{shares}{ticker}{pps}D{buyDate}$--{self.capital}")
            return True
        else: 
            print("insufficient capital to complete buy transaction")
            self.bankrupt = True   
            return False 
    
    def sell(self, pps, iD, ticker, sellDate): 
        if ticker in self.trades: 
            for transaction in self.trades[ticker]: 
                if transaction.contractID == iD: 
                    shares = transaction.num 
                    self.capital += shares * pps
                    self.trades[ticker].remove(transaction) 
                    print(f"S{shares}{ticker}{pps}D{sellDate}$--{self.capital}")
                    break

    def clearTradeTick(self, date, iD, ticker): 
        """ clears unsold ticker asset (id) from the book using the intial purchase price 
        """
        if ticker in self.trades: 
            for transaction in self.trades[ticker]:
                if transaction.contractID == iD: 
                    self.capital += transaction.num*transaction.pps
                    self.valuation.append(self.capital)
                    self.trades[ticker].remove(transaction)
                    print(f"FS{transaction.num}{ticker}{transaction.pps}D{date}$--{self.capital}")
                    break

In [8]:
def getPricePerShare(p: BenchmarkPortfolio, t: str, date: str, id:str, isSale: bool):
    """ filters API calls to return the pps of the stock 45 minutes before closing on a given date, only 
        used for buy orders 
        requires: date is a us Trading Day

        @p: the portfolio object that is making the trade
        @t: stock ticker 
        @date: date of the trade
        @id: contractID triggering the trade 
        @isSale: flag to determine if the trade is a buy or sell
    """
    numtries = 0
    
    try:
        while numtries <= 1:
            response = requests.get(url=setLinkIntd(t, date, date, "5min"))

            if response.status_code == 200:
                jsonData = response.json()
                if len(jsonData) > 15:
                    pps = jsonData[-15]["open"]
                    return pps
                elif len(jsonData) == 0:
                    response = requests.get(url=setLinkEod(t, date, date))
                    try:
                        pps = response.json()['historical'][0]['close']
                    except KeyError as e:
                        p.logError(e, t, date, id, isSale, "getPricePerShare")
                        return None
                    except Exception as e:
                        p.logError(e, t, date, id, isSale, "getPricePerShare")
                        return None
                return pps
            elif response.status_code == 429:
                time.sleep(60*(numtries+1))
                print(f"Too many API calls exception, waiting until minute reset. current attempt: {numtries}")
                numtries += 1
            else:
                p.logError(response.status_code, t, date, id, isSale, "getPricePerShare")
                return None
    except Exception as e: 
            p.logError(e, t, date, id, isSale, "getPricePerShare")
            return None 

In [9]:
def getHistorical(p:BenchmarkPortfolio, t:str, s:str, e:str, f:str, id) -> pd.DataFrame:
    """ returns a pd.Dataframe of the opening prices of the stock over a given time period [s, e] 

        @p: the portfolio object making a trade 
        @t: stock ticker of the underlying
        @s: starting period
        @e: end 
        @f: frequency of price observations i.e. 5min, 15min etc...
        @id: contract id triggering the trade
    """
    
    numtries = 0

    try:
        while numtries <=1: 
            response = requests.get(url=setLinkIntd(t, s, e, f))
            if response.status_code == 200: 
                try:
                    data = response.json() or requests.get(setLinkEod(t, s, e)).json().get(('historical'), [])
                except Exception as e: 
                    p.logError(e, t, s, id, True, "getHistorical")
                    return pd.DataFrame()
                if len(data)!= 0:
                    prices = pd.DataFrame([entry['close'] for entry in data])
                    return prices 
            elif response.status_code == 429: 
                time.sleep(60*(numtries+1))
                print(f"Too many API calls exception, waiting until minute reset. current attempt: {numtries}")
                numtries +=1
            else: 
                p.logError(response.status_code, t, s, id, True, "getHistorical")
                return pd.DataFrame()              
    except Exception as e: 
        p.logError(e, t, s, id, True, "getHistorical")
        return pd.DataFrame() 

In [10]:
def getEMA (prices: pd.DataFrame, period:int):
    return prices.ewm(span=period, adjust=False).mean()

In [11]:
def getMACD(priceDf: pd.DataFrame, sP:int, fP:int, sigP:int):
    """ Computes the (12, 26, 9) MACD for the underlying to time entry positions.
        Requires: len(priceDf) >= slow period + signal period 
    """
    # assert len(priceDf) >= (sP + sigP), f"price_data must have at least {sP + sigP} data points for cell 13: getMACD, it currently has {len(priceDf)}"
    
    emaFast = getEMA(priceDf, fP)
    emaSlow = getEMA(priceDf, sP)
    macdLine = emaFast - emaSlow
    signalLine = getEMA(macdLine, sigP)

    histogram = macdLine - signalLine
    if histogram.iloc[-1].item() > 0.1:
        return True
    else:
        return False


In [12]:
def getBuyDayMACD(p:BenchmarkPortfolio, date:str, tick:str, id:str, slow:int, fast:int, signal:int):
    """ recursively times the entry date of {tick} using a MACD divergence signal (histogram >0.1)

        @p: portfolio executing the trade
        @date: current date 
        @tick: ticker of the underlying stock to purchase  
        @id: contract ID signalling the purchase 
        @slow: MACD tralling slow period
        @fast: MACD trailling fast period
        @signal: MACD sma period 
    """
    thresh = slow+ signal
    curr = pd.Timestamp(getTradingDay(date))
    minPeriod = curr - (thresh*usBday)

    prices = getHistorical(p, tick, minPeriod.strftime('%Y-%m-%d'), curr.strftime('%Y-%m-%d'), "1day", id)

    if prices is None or prices.empty:
        return None
    if len(prices) >= thresh:
        prices = prices.iloc[::-1].reset_index(drop=True)
        if getMACD(prices, slow, fast, signal):
            return curr.strftime('%Y-%m-%d')
        else:
            date = curr + usBday
            return getBuyDayMACD(p, date, tick, id, slow, fast, signal)
    else:
        return None  

In [13]:
def meanReversion(prices: pd.Series, window:int, threshold:int):
    try:
        if len(prices) < window:
            raise ValueError(f"Warning: Not enough data points for rolling window (got {len(prices)}, expected {window})")
    except ValueError as e: 
        if (len(prices>15)):
            window = len(prices)
        else:
            return False
            
    sma = prices.rolling(window=window).mean()
    std_dev = prices.rolling(window=window).std()

    z_score = (prices - sma) / std_dev

    if len(z_score) > 0:
        if z_score.iloc[-1] > threshold:
            return True  
        elif z_score.iloc[-1] < -threshold:
            return True
    else:
        print("Error: 'z_score' is empty or invalid.")
    
    return False  

In [14]:
def getSellDay(contractDate:str, ticker:str, window:int, thresh:int, call_count=0):
    if call_count >= 365:
        print("Reached maximum recursion mean reversion limit of 365 days. returning today's date to force sell")
        return None

    adjDate = pd.to_datetime(getTradingDay(contractDate))
    
    startDay = adjDate - pd.Timedelta(days=30)
    endDay = adjDate - pd.Timedelta(days=1)
    
    response = requests.get(setLinkIntd(ticker, startDay.strftime('%Y-%m-%d'), endDay.strftime(('%Y-%m-%d')), "5min"))

    if response.status_code == 200:
        data = response.json()
        if len(data) == 0: 
            try:
                response = requests.get(setLinkEod(ticker, startDay.strftime('%Y-%m-%d'), endDay.strftime(('%Y-%m-%d'))))
                data = response.json()['historical']
            except Exception as e: 
                print("Unexpected exception: "+ str(e) + " at cell 12: getSellDay()")
                return None
        if len(data) != 0:
            prices = pd.Series([entry['close'] for entry in data]).iloc[::-1]
            
            if meanReversion(prices, window, thresh):
                return adjDate.strftime('%Y-%m-%d')  
            else:
                next_day = adjDate + usBday
                return getSellDay(next_day.strftime('%Y-%m-%d'), ticker, window, thresh, call_count + 1)  
        else: 
            return None
    else:
        print(f"Error fetching data: {response.status_code} at cell 12: getSellDay(). Returning default sell day")
        return adjDate


In [15]:
def executeOrder(portfolio: Portfolio, benchPortfolio: BenchmarkPortfolio, w: float, ticker: str, date: str, id: int, isSale: bool) -> bool:
    """ executes a buy or sell order on the portfolio and benchmark portfolio 

        @portfolio: the strategy's portfolio to execute the order on 
        @benchPortfolio: the benchmark SPX portfolio to execute the order on 
        @w: the percentage of capital to allocate to the trade 
        @ticker: the ticker of the underlying asset 
        @date: the date of the trade 
        @id: the contractID triggering the trade 
        @isSale: flag to determine if the trade is a buy or sell
    """
    
    pps = getPricePerShare(portfolio, ticker, date, id, isSale)
    #ppsSPX = getPricePerShare(benchPortfolio, "^SPX", date, id, isSale)

    if (pps!=None): 
        if not isSale:
            if (portfolio.isClear and benchPortfolio.isClear):
                shares = math.floor((w*portfolio.capital)/pps)
                #sharesSPX = math.floor((w*benchPortfolio.capital)/ppsSPX)
                portfolio.buy(ticker, pps, shares, id, date)
                #benchPortfolio.buy(ppsSPX, sharesSPX, id, date)            
                return True 
            else:
                return False 
        else: 
            portfolio.sell(pps, id, ticker, date)
            #benchPortfolio.sell(pps, id, date)
            return True 
    return False 

In [16]:
"""temp = Portfolio(2.5, 0.1, 1000)
print(getBuyDayMACD(temp, "2016-01-01", "LMT", "0000", 26, 18, 9))"""

'temp = Portfolio(2.5, 0.1, 1000)\nprint(getBuyDayMACD(temp, "2016-01-01", "LMT", "0000", 26, 18, 9))'

In [17]:
def getRatios(p:BenchmarkPortfolio, tick: str, date: str):
    """ returns a dict of the closest quarter reported financial ratios of a given ticker on some date, none if 
        api call was unsuccessful 
        note: since historicRatios are ordered from most recent descending, the first instance where historicRatios < date
              will be the most recent quarter report
            
        @tick: the ticker symbol of the company
        @date: dathe date for which the financial ratios are required 
    """
    date = pd.to_datetime(date)
    response = requests.get(setLinkRatios(tick)) 
    if response.status_code == 200:
        historicRatios = response.json() or []
        for ratio in historicRatios:
            currentQuarter = pd.to_datetime(ratio['date'])
            if currentQuarter <= date:
                p.cacheIndicator(FundementalIndicator.FINANCIAL_RATIOS, ratio, tick)
                return ratio
    return None


In [18]:
def getIncomeStatement (p:BenchmarkPortfolio, tick:str, date:str):

    date = pd.to_datetime(date)
    response = requests.get(setLinkIncomeStatement(tick))
    if response.status_code ==200:
        historicIS = response.json()
        for statement in historicIS:
            currentQuarter = pd.to_datetime(statement['date'])
            if currentQuarter <= date: 
                p.cacheIndicator(FundementalIndicator.INCOME_STATEMENT, statement, tick)
                return statement
    return None

In [19]:
def compositeMargins(ratioDict:dict, w1:int, w2:int, w3:int, w4:int):
    comp = (w1*ratioDict['grossProfitMargin'] + 
            w2*ratioDict['operatingProfitMargin'] + 
            w3*ratioDict['pretaxProfitMargin'] + 
            w4*ratioDict['netProfitMargin'])
    return comp

In [20]:
def compositeCashFlow(ratioDict:dict, w1:int, w2:int, w3:int, w4:int):
    comp = (w1*ratioDict['operatingCashFlowPerShare'] + 
            w2*ratioDict['freeCashFlowPerShare'] + 
            w3*ratioDict['operatingCashFlowSalesRatio'] + 
            w4*ratioDict['cashPerShare'])
    return comp

In [21]:
def compositeReturns(ratioDict:dict, w1:int, w2:int, w3:int):
    comp = (w1*ratioDict['returnOnAssets'] + 
            w2*ratioDict['returnOnEquity'] + 
            w3*ratioDict['returnOnCapitalEmployed'])
    return comp

In [22]:
def compositeOperatingCycle(ratioDict:dict, w1:int, w2:int, w3:int):
    comp = (w1*ratioDict['daysOfInventoryOutstanding'] + 
            w2*ratioDict['operatingCycle'] + 
            w3*ratioDict['cashConversionCycle'])
    return comp

In [23]:
def compositeValuation(ratioDict:dict, w1:int, w2:int, w3:int, w4:int):
    comp = (w1*ratioDict['priceEarningsRatio'] + 
            w2*ratioDict['priceToSalesRatio'] + 
            w3*ratioDict['priceToBookRatio'] + 
            w4*ratioDict['priceToFreeCashFlowsRatio'])
    return comp

In [24]:
def grossProfitToContract(incomeStatement:dict, quarterlyCV):
    comp = incomeStatement['grossProfitRatio']*(quarterlyCV)
    return comp

In [25]:
def deltaEbitda(incomeStatement: dict, quarterlyCV):
    """ estimates how a new contract affects EBITA using contract revenue x EBITDA margin
    """
    comp = quarterlyCV*incomeStatement['ebitda']/incomeStatement['revenue']
    return comp

In [26]:
def calculateTradeAllocation(p:BenchmarkPortfolio, tick:str, date:str, cv:int, numContracts:int, minCap=0.1, maxCap=0.5):
    rDf = p.getIndicatorReport(FundementalIndicator.FINANCIAL_RATIOS, tick, date) or getRatios(p, tick, date)
    incomeDf = p.getIndicatorReport(FundementalIndicator.INCOME_STATEMENT) or getIncomeStatement(p, tick, date)

    if rDf is not None:
        cM = compositeMargins(rDf, 0.1, 0.4, 0.1, 0.4)
        cCF = compositeCashFlow(rDf, 0.4, 0.25, 0.25, 0.1)
        cR = compositeReturns(rDf, 0.2, 0.5, 0.3)
        cOC = compositeOperatingCycle(rDf, 0.1, 0.6, 0.3)
        cV = compositeValuation(rDf, 0.4, 0.2, 0.1, 0.3)
    if incomeDf is not None: 
        cPTC = grossProfitToContract(incomeDf, cv/4.0)


In [27]:
def closePortfolio(p1: BenchmarkPortfolio, p2: BenchmarkPortfolio):
    """ closes the portfolios and return their performances after blacktesting over some period FY-{X}
        asserts that no pending market positions need to be resolved
    """
    results = []
    results.append(p1)
    results.append(p2)
    assert p1.holds == False, f"there are positions waiting to be resolved on {p1}"
    assert p2.holds == False, f"there are positions waiting to be resolved on {p1}"
    assert len(p1.trades) == 0, f"there are {len(p1.trades)} positions waiting to be closed on {p2}"
    assert len(p2.trades) == 0, f"there are {len(p2.trades)} positions waiting to be closed on {p2}"
    return results

In [28]:
strategyPortfolio = Portfolio(2.5, 0.1, 1000000000)
spxBenchmark = BenchmarkPortfolio(1000000000, 0.1)

In [29]:
def runPortfolio(contracts, z, w, c, bSig, window): 
    """ runs the portfolio simulation on the given contracts 

        @contracts: dataframe of contracts 
        @z: mean reversion z-score treshold
        @w: weight of total capital to allocate to each trade 
        @c: initial capital 
        @bSig: days before contract start date to buy
    """
    global strategyPortfolio, spxBenchmark

    for i in range (len(contracts)):
        currContract = contracts.iloc[i]
        tick = currContract['Recipient Name']
        contractStart = currContract['Start Date'] # type(contractStart) = pd.Timestamp
        id = currContract['internal_id']
        factor = currContract['Weight']
        buyDay = getBuyDayMACD(strategyPortfolio, contractStart, tick, id, 26, 18, 9) or getBuyDay(contractStart, bSig)
        if buyDay == None:
            buyDay = getBuyDay(contractStart, 3)
        allocation = w*factor
        if (executeOrder(strategyPortfolio, spxBenchmark, allocation, tick, buyDay, id, False)): 
            sellDay = getSellDay(buyDay, tick, window, z)
            if (sellDay != None):
                if (executeOrder(strategyPortfolio, spxBenchmark, w, tick, sellDay, id, True)):
                    strategyPortfolio.sellSuccessful()
                    #spxBenchmark.sellSuccessful()
                else: 
                    strategyPortfolio.clearTradeTick(buyDay, id, tick)
                    #spxBenchmark.clearTradeSPY(id, buyDay)
            else:
                strategyPortfolio.clearTradeTick(buyDay, id, tick)
                #spxBenchmark.clearTradeSPY(id, buyDay)
        else:
            continue 
    closePortfolio(strategyPortfolio, spxBenchmark)


In [34]:
temp = runPortfolio(contracts, 2.5, 0.1, 100000000, 3, 20)

B3807127LMT463.12D2023-01-27$--195906696.0658009
S3807127LMT477.53D2023-02-21$--2013924052.3758008
B1111600NOC543.52D2023-01-03$--1409747220.3758008
S1111600NOC442.3741D2023-01-20$--1901490269.9358008
B5603330GE67.87D2023-01-04$--1521192262.8358006
S5603330GE78.19D2023-01-12$--1959316635.5358007
B3880987RTX100.97D2023-02-13$--1567453378.1458006
S3880987RTX100.87D2023-02-15$--1958928536.8358006
B3806865LMT463.12D2023-01-27$--195893218.0358007
S3806865LMT477.53D2023-02-21$--2013785461.4858005
B3913471LMT463.12D2023-01-27$--201378771.96580052
S3913471LMT477.53D2023-02-21$--2070178578.5958004
B665502CACI311.07D2023-01-04$--1863160871.4558005
S665502CACI309.95D2023-02-01$--2069433216.3558006
B1142239NOC543.52D2023-01-03$--1448603475.0758007
S1142239NOC442.3741D2023-01-20$--1953900424.6857007
B2779111LHX210.92D2023-01-27$--1367730332.5657005
S2779111LHX209.2835D2023-02-07$--1949352409.5342007
B3987221LDOS97.78D2023-02-02$--1559481940.1542006
S3987221LDOS101.37D2023-02-13$--1963666532.9242005

AssertionError: there are 103 positions waiting to be closed on <__main__.BenchmarkPortfolio object at 0x75bc9b9346e0>

In [44]:
temp = strategyPortfolio.valuation

fy19 = temp[:673]
fy23 = temp[672:]

In [50]:
df23=pd.DataFrame(fy23)
df19=pd.DataFrame(fy19)
df19.to_csv("R2 returns/trades/2019.csv", index=False)
df23.to_csv("R2 returns/trades/2023.csv", index=False)